In [3]:
# 1st party imports
from datetime import date, datetime, timedelta
import gzip
import io

# 3rd party imports
import boto3
from botocore import UNSIGNED
from botocore.config import Config
# import pandas as pd
import polars as pl

# pd.options.mode.chained_assignment = None



In [4]:
S3_BUCKET = "ooni-data-eu-fra"
S3_PREFIX = "raw"

def build_prefix(
    timestamp: datetime,
    cc: str,
    test: str = "webconnectivity",
) -> str:
    """Build the S3 prefix"""
    yyyy = f"{timestamp.year:04d}"
    mm = f"{timestamp.month:02d}"
    dd = f"{timestamp.day:02d}"
    hh = f"{timestamp.hour:02d}"
    cc = cc.upper()
    test = test.lower()
    # return f"s3://{S3_BUCKET}/{S3_PREFIX}/{yyyy}{mm}{dd}/{hh}/{cc}/{test}/"
    return f"{S3_PREFIX}/{yyyy}{mm}{dd}/{hh}/{cc}/{test}/"



In [24]:
ts = datetime(year=2023, month=1, day=5, hour=10)
s3_prefix = build_prefix(ts, "ru")

aws_config = Config(signature_version=UNSIGNED)
# s3 = boto3.client('s3', config=aws_config)
# objs = s3.list_objects_v2(Bucket=S3_BUCKET, Prefix=s3_prefix)

s3 = boto3.resource('s3', config=aws_config)
s3_bucket = s3.Bucket(S3_BUCKET)
objs = s3_bucket.objects.filter(Prefix=s3_prefix)
objs = filter(lambda o: o.key.endswith(".jsonl.gz"), objs)

o = next(objs)

# for o in objs:
#     print(f"s3://{o.bucket_name}/{o.key}")

buf = io.BytesIO(gzip.decompress(o.get()['Body'].read()))


s3://ooni-data-eu-fra/raw/20230105/10/RU/webconnectivity/2023010510_RU_webconnectivity.n1.0.jsonl.gz
s3://ooni-data-eu-fra/raw/20230105/10/RU/webconnectivity/2023010510_RU_webconnectivity.n1.1.jsonl.gz
s3://ooni-data-eu-fra/raw/20230105/10/RU/webconnectivity/2023010510_RU_webconnectivity.n1.2.jsonl.gz


In [27]:
# df = pl.scan_ndjson(
#     buf,
#     n_rows=4,
#     low_memory=False,
# )

# lines = buf.readlines()

# len(lines)

local_file = "/home/max/workspace/de/maxresing/ooni-dashboard/data/2023010510_RU_webconnectivity.n1.0.jsonl"
df = pl.read_ndjson(
    local_file
)

df
    





thread '<unnamed>' panicked at 'index out of bounds: the len is 0 but the index is 0', /home/runner/work/polars/polars/polars/polars-io/src/ndjson_core/buffer.rs:171:41


PanicException: index out of bounds: the len is 0 but the index is 0

In [ ]:

df = pl.read_json(
    buf,
    json_lines=True,
).lazy()

"""
df = pd.read_json(
    o.get()['Body'],
    lines=True,
    compression='gzip',
)
df = df[0:3]
"""
df